In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
books = pd.read_csv('books_meta_sample.csv', on_bad_lines='skip', encoding="UTF-8")
books.columns = ['book_id', 'name', 'link', 'author']

In [3]:
users = pd.read_csv('users.csv', on_bad_lines='skip', encoding="UTF-8")
users.columns = ['user_id', 'join_date', 'age', 'gender']

In [4]:
ratings = pd.read_csv('interactions.csv', on_bad_lines='skip', encoding="UTF-8")
ratings.columns = ['user_id', 'book_id', 'rating','timestamp', 'review_length_chars', 'liked']

In [5]:
print(ratings.shape)
print(list(ratings.columns))

(11920, 6)
['user_id', 'book_id', 'rating', 'timestamp', 'review_length_chars', 'liked']


In [6]:
combine_book_rating = pd.merge(ratings, books, on='book_id')
columns = ['link', 'author', 'timestamp', 'review_length_chars', 'liked']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
print(combine_book_rating.head())

  user_id  book_id  rating                               name
0    U054     1418       4  Thay Đổi Tất Cả Chỉ Trừ Vợ Và Con
1    U064      526       5           Hãy Gọi Tên Tôi (Hồi Ký)
2    U041      310       4             CHUYỆN ĐỜI CHUYỆN NGHỀ
3    U004     1015       4            Vùng Đất Quỷ Tha Ma Bắt
4    U028     1015       4            Vùng Đất Quỷ Tha Ma Bắt


In [7]:
book_ratingCount = (combine_book_rating.
     groupby(by = ['name'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['name', 'totalRatingCount']]
    )
print(book_ratingCount.head())

                                                name  totalRatingCount
0                     Kho Tàng Truyện Cười Việt Nam                  3
1   Thiết Kế Tổ Chức (Organizational Design) - Ri...                 4
2                  Tuyển Tập Du Ký( Tặng kèm sổ tay)                 2
3                             "Đức Phật" nơi công sở                 1
4  (In màu toàn bộ, Combo 2 tập) MA QUỶ DÂN GIAN ...                 1


In [8]:
rating_with_totalRatingCount = combine_book_rating.merge(
    book_ratingCount, 
    left_on = 'name', 
    right_on = 'name', 
    how = 'left')
print(rating_with_totalRatingCount.head())

  user_id  book_id  rating                               name  \
0    U054     1418       4  Thay Đổi Tất Cả Chỉ Trừ Vợ Và Con   
1    U064      526       5           Hãy Gọi Tên Tôi (Hồi Ký)   
2    U041      310       4             CHUYỆN ĐỜI CHUYỆN NGHỀ   
3    U004     1015       4            Vùng Đất Quỷ Tha Ma Bắt   
4    U028     1015       4            Vùng Đất Quỷ Tha Ma Bắt   

   totalRatingCount  
0                99  
1                16  
2                 1  
3                16  
4                16  


In [9]:
print(book_ratingCount['totalRatingCount'].describe())

count    1287.000000
mean        9.261849
std        20.010082
min         1.000000
25%         1.000000
50%         2.000000
75%         6.000000
max       100.000000
Name: totalRatingCount, dtype: float64


In [10]:
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

0.90     22.00
0.91     26.26
0.92     32.00
0.93     35.98
0.94     43.68
0.95     55.70
0.96     70.12
0.97     88.00
0.98     99.00
0.99    100.00
Name: totalRatingCount, dtype: float64


In [11]:
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount[
    rating_with_totalRatingCount['totalRatingCount'] >= popularity_threshold
]
print(rating_popular_book.head())

   user_id  book_id  rating  \
0     U054     1418       4   
9     U043      176       4   
16    U007      786       4   
17    U005      570       4   
18    U095     1481       3   

                                                 name  totalRatingCount  
0                   Thay Đổi Tất Cả Chỉ Trừ Vợ Và Con                99  
9       Combo Chuyện Đông Chuyện Tây (Trọn Bộ 4 Cuốn)                50  
16                  Sáu Kẻ Nói Dối Và Lá Thư Buộc Tội               100  
17                                Rừng Nauy (Tái Bản)                61  
18  Warren Buffett - Quá Trình Hình Thành Một Nhà ...               100  


In [21]:
from scipy.sparse import csr_matrix

rating_popular_book_pivot = rating_popular_book.pivot(
    index='name',
    columns='user_id',
    values='rating'
).fillna(0)

rating_popular_book_matrix = csr_matrix(rating_popular_book_pivot.values)

print(rating_popular_book_pivot.head())

user_id                              U001  U002  U003  U004  U005  U006  U007  \
name                                                                            
451 Độ F                              5.0   3.0   0.0   0.0   4.0   5.0   0.0   
60 Năm Văn Học Đương Đại Trung Quốc   0.0   3.0   0.0   4.0   0.0   4.0   3.0   
Anna Karenina - Tập 1                 0.0   4.0   5.0   0.0   4.0   4.0   4.0   
Bách hóa giấc mơ của ngài Dollargut   4.0   5.0   4.0   3.0   4.0   4.0   4.0   
Bút Hết Nặng Viết Hết Đau             4.0   4.0   3.0   4.0   3.0   4.0   3.0   

user_id                              U008  U009  U010  ...  U091  U092  U093  \
name                                                   ...                     
451 Độ F                              5.0   4.0   4.0  ...   0.0   0.0   0.0   
60 Năm Văn Học Đương Đại Trung Quốc   0.0   0.0   3.0  ...   4.0   5.0   4.0   
Anna Karenina - Tập 1                 0.0   5.0   0.0  ...   4.0   5.0   4.0   
Bách hóa giấc mơ của ngài Dollar

In [13]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(rating_popular_book_matrix)

print(model_knn)

NearestNeighbors(algorithm='brute', metric='cosine')


In [14]:
query_index = np.random.choice(rating_popular_book_pivot.shape[0])
print(query_index)
print(rating_popular_book_pivot.iloc[query_index,:].values.reshape(1,-1))
distances, indices = model_knn.kneighbors(rating_popular_book_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
rating_popular_book_pivot.index[query_index]

33
[[5. 4. 3. 4. 4. 4. 4. 3. 3. 3. 3. 3. 3. 4. 0. 3. 4. 4. 4. 3. 5. 3. 4. 3.
  4. 4. 4. 5. 4. 2. 3. 3. 3. 3. 4. 4. 5. 4. 4. 4. 3. 2. 4. 4. 3. 3. 2. 5.
  3. 3. 5. 4. 4. 4. 5. 4. 4. 4. 4. 3. 3. 3. 3. 4. 2. 4. 3. 4. 3. 3. 4. 4.
  5. 3. 3. 4. 4. 4. 4. 3. 4. 5. 3. 4. 5. 4. 3. 3. 3. 4. 4. 5. 3. 4. 3. 3.
  3. 4. 3. 3.]]


'Những Vệ Binh Vương Quốc - Disney Lúc Rạng Đông ( Tập 2 )'

In [15]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_popular_book_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_book_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Những Vệ Binh Vương Quốc - Disney Lúc Rạng Đông ( Tập 2 ):

1: Tây Du Hí  - Tập 6 (Tặng Kèm 2 Thẻ Bài), with distance of 0.025487095687328987:
2: Peter Drucker - Tư Duy Đúng Quản Lý Đúng, with distance of 0.02568807622589353:
3: Đi qua hoa cúc (Tái Bản 2022), with distance of 0.026677367312259448:
4: Những Chàng Trai Ở Lộc Phong Quán – Tập 14, with distance of 0.0267348810761463:
5: Đệ Tử Quy (Tái Bản), with distance of 0.029365553998886296:


In [16]:
import unicodedata
from difflib import get_close_matches

def normalize_text(s):
    """Chuẩn hóa chuỗi tiếng Việt: bỏ dấu, chữ thường"""
    s = s.lower().strip()
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')  # bỏ dấu
    return s

def recommend_books_vi(query, model_knn, pivot_table, n_recommendations=5, min_similarity=0.6):
    """
    Gợi ý sách tiếng Việt có hỗ trợ fuzzy matching, bỏ dấu, 
    và tìm theo cụm từ nếu đầu vào quá ngắn.
    """
    # Chuẩn hóa đầu vào
    query_norm = normalize_text(query)
    words = query_norm.split()
    
    # Chuẩn hóa tất cả tên sách
    titles = pivot_table.index.tolist()
    titles_norm = [normalize_text(t) for t in titles]
    title_map = dict(zip(titles_norm, titles))

    # --- Nếu người dùng nhập ngắn (1–2 từ), tìm tất cả tựa chứa cụm đó ---
    if len(words) <= 2:
        matches = [title_map[t] for t in titles_norm if query_norm in t]
        if not matches:
            return f"Không tìm thấy tựa sách nào chứa cụm '{query}'."
        print(f"Tìm thấy {len(matches)} tựa có chứa cụm '{query}':")
        for i, m in enumerate(matches[:5]):
            print(f"{i+1}. {m}")
        # Chọn sách đầu tiên để tiếp tục gợi ý
        matched_title = matches[0]
        print(f"\nSử dụng sách '{matched_title}' để gợi ý tương tự...\n")

    # --- Nếu tên dài hơn, dùng fuzzy match ---
    else:
        close_matches = get_close_matches(query_norm, titles_norm, n=1, cutoff=min_similarity)
        if not close_matches:
            return f"Không tìm thấy sách nào gần giống với '{query}'."
        matched_title = title_map[close_matches[0]]
        print(f"Tìm thấy gần nhất: '{matched_title}'\n")

    # --- Gợi ý các sách tương tự ---
    book_vector = pivot_table.loc[matched_title].values.reshape(1, -1)
    distances, indices = model_knn.kneighbors(book_vector, n_neighbors=n_recommendations + 1)

    print(f"Gợi ý cho sách: '{matched_title}'\n")
    for i in range(1, len(distances.flatten())):
        similar_book = pivot_table.index[indices.flatten()[i]]
        similarity = 1 - distances.flatten()[i]
        print(f"{i}. {similar_book} (Độ tương đồng: {similarity:.3f})")

In [20]:
query = input("Nhập tên (hoặc một phần tên) sách tiếng Việt: ")
recommend_books_vi(query, model_knn, rating_popular_book_pivot)

Tìm thấy gần nhất: 'Đệ Tử Quy (Tái Bản)'

Gợi ý cho sách: 'Đệ Tử Quy (Tái Bản)'

1. Đi qua hoa cúc (Tái Bản 2022) (Độ tương đồng: 0.991)
2. Tây Du Hí  - Tập 6 (Tặng Kèm 2 Thẻ Bài) (Độ tương đồng: 0.990)
3. Sách Tiếng Việt - Văn Việt - Người Việt (Tái bản năm 2021) (Độ tương đồng: 0.985)
4. Peter Drucker - Tư Duy Đúng Quản Lý Đúng (Độ tương đồng: 0.984)
5. Thánh Kinh Marketing (Độ tương đồng: 0.984)
